In [118]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import random
import folium

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def randomColor():
    color = "#"+"".join(random.choice("0123456789ABCDEF") for i in range(6))
    return color

def jitter(dataFrame):
    rng = np.random.default_rng(seed=1)
    jitter_strength = 1000

    dataFrame["TotalPopJitter"] = dataFrame["Total Population"] + rng.normal(0, jitter_strength, size = len(dataFrame))
    return dataFrame.sort_values(by = "TotalPopJitter", ascending = False).reset_index(drop = True)

tractLocsPops = gpd.read_file("tractLocsPops.geojson")
#tractLocsPops = tractLocsPops.sort_values(by = "Total Population", ascending = False).reset_index(drop=True)
tractLocsPops = jitter(tractLocsPops)
tractLocsPops["cluster"] = None

clusterScaleFactor = 0.05#float(input("Cluster Scale Factor: "))
clusterIndex = -1
clusterColors = []

for i in range(len(tractLocsPops)):
    lat0 = tractLocsPops.iloc[i]["INTPTLAT"]
    lon0 = tractLocsPops.iloc[i]["INTPTLON"]
    tractLocsPops["distance"] = haversine(lat0, lon0, tractLocsPops["INTPTLAT"],tractLocsPops["INTPTLON"])
        
    clusterScale = clusterScaleFactor*np.sqrt(tractLocsPops.iloc[i]["Total Population"])
    clustered = pd.notna(tractLocsPops.iloc[i]["cluster"])
    
    if (not clustered) & tractLocsPops.loc[tractLocsPops["distance"] <= clusterScale, "cluster"].isna().all():
        clusterIndex += 1
        clusterColors.append(randomColor())
        tractLocsPops.loc[(tractLocsPops["distance"] <= clusterScale) & (tractLocsPops["cluster"].isna()), "cluster"] = clusterIndex
    elif not clustered:
        nearestCluster = tractLocsPops[
            (tractLocsPops["distance"] <= clusterScale) &
            (tractLocsPops["cluster"].notna())
        ].nsmallest(1, "distance")["cluster"].squeeze()
        tractLocsPops.loc[(tractLocsPops["distance"] <= clusterScale) & (tractLocsPops["cluster"].isna()), "cluster"] = nearestCluster
    else:
        tractLocsPops.loc[(tractLocsPops["distance"] <= clusterScale) & (tractLocsPops["cluster"].isna()), "cluster"] = tractLocsPops.iloc[i]["cluster"]

tractLocsPops = tractLocsPops.drop(columns = ["distance"])
tractLocsPops["cluster"] = tractLocsPops["cluster"].astype(int)
    
# ax = tractLocsPops.plot(figsize = (10,10), color = [clusterColors[i] for i in tractLocsPops["cluster"]])
# plt.show()

def tractStyle(feature):
    cluster = feature["properties"]["cluster"]
    color = clusterColors[cluster]
    return {
        "fillColor": color,
        "fillOpacity": 0.8
    }

m = folium.Map(location=[37.7749, -122.4194], zoom_start=8)

folium.GeoJson(
    tractLocsPops,
    style_function = tractStyle
).add_to(m)
m.save("TractMiniClusters.html")